# Mask R-CNN demo

This notebook illustrates one possible way of using `maskrcnn_benchmark` for computing predictions on images from an arbitrary URL.

Let's start with a few standard imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

import requests
from io import BytesIO
from PIL import Image
import numpy as np

from torchvision import transforms as T
import torch
import cv2

import os
import dense_correspondence_manipulation.utils.utils as pdc_utils
from dense_correspondence_manipulation.maskrcnn_tools.inference import MaskRCNNInference

In [ ]:
# this makes our figures bigger
pylab.rcParams['figure.figsize'] = 20, 12

Those are the relevant imports for the detection model

In [ ]:
from maskrcnn_benchmark.config import cfg
from predictor import COCODemo

We provide a helper class `COCODemo`, which loads a model from the config file, and performs pre-processing, model prediction and post-processing for us.

We can configure several model options by overriding the config options.
In here, we make the model run on the CPU

In [ ]:
# config_file = os.path.join(pdc_utils.getDenseCorrespondenceSourceDir(), "external/maskrcnn-benchmark/configs",
#                            "caffe2/e2e_mask_rcnn_R_50_FPN_1x_caffe2.yaml")

# config_file = os.path.join(pdc_utils.getDenseCorrespondenceSourceDir(), "config/maskrcnn/e2e_mask_rcnn_R_101_FPN_1x_SINGLE_GPU.yaml")

# # update the config options with the config file
# cfg.merge_from_file(config_file)
# # manual override some options
# cfg.merge_from_list(["MODEL.DEVICE", "cuda"])
# cfg.MODEL.WEIGHT = "/home/manuelli/data/pdc/trained_models/maskrcnn/shoes/shoes_101_FPN/model_final.pth"

model_folder = "/home/manuelli/data/pdc/trained_models/maskrcnn/shoes/shoes_101_FPN"
inference = MaskRCNNInference.from_model_folder(model_folder)

Now we create the `COCODemo` object. It contains a few extra options for conveniency, such as the confidence threshold for detections to be shown.

Let's define a few helper functions for loading images from a URL

In [ ]:
def load(url):
    """
    Given an url of an image, downloads the image and
    returns a PIL image
    """
    response = requests.get(url)
    pil_image = Image.open(BytesIO(response.content)).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

def imshow(img):
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.axis("off")
    
def get_image():
    """
    Import image from pdc dataset
    """
    scene_name = "2018-11-16-21-00-00"
    images_path = os.path.join(pdc_utils.get_data_dir(), 'logs_proto', "2018-11-16-21-00-00", "processed/images")
    img_idx = 0
    
#     image_filename = "/home/manuelli/data/pdc/coco/shoes/multi_shoe_train_10k/images/00000.png"
    image_filename = os.path.join(images_path, '{0:06d}_rgb.png'.format(img_idx))
    pil_image = Image.open(image_filename).convert("RGB")
    # convert to BGR format
    image = np.array(pil_image)[:, :, [2, 1, 0]]
    return image

Let's now load an image from the COCO dataset. It's reference is in the comment

In [ ]:
# from http://cocodataset.org/#explore?id=345434
# image = load("http://farm3.staticflickr.com/2469/3915380994_2e611b1779_z.jpg")
image = get_image()
imshow(image)

### Computing the predictions

We provide a `run_on_opencv_image` function, which takes an image as it was loaded by OpenCV (in `BGR` format), and computes the predictions on them, returning an image with the predictions overlayed on the image.

In [ ]:
# compute predictions
predictions = inference.coco_demo.run_on_opencv_image(image)
imshow(predictions)

print(type(predictions))
print(predictions.shape)

### Saving predictions to disk
Save the predictions to a directory

In [ ]:
predictions = inference.coco_demo.compute_prediction(image)
print(type(predictions))

save_dir = os.path.join(pdc_utils.getDenseCorrespondenceSourceDir(), "scratch/maskrcnn_predictions")
MaskRCNNInference.save_predictions(save_dir, predictions)

### Visualize Mask

In [ ]:
# visualize the saved mask
data = pdc_utils.getDictFromYamlFilename(os.path.join(save_dir, 'data.yaml'))
data['detections']
mask_img = Image.open(os.path.join(save_dir, data['detections'][0]['mask_image_filename']))
plt.imshow(np.asarray(mask_img))